# Tutorial to Work with a GOES-R (Geostationary) ABI Level 2 Data File

This tutorial was written in December 2022 by Dr. Rebekah Esmaili, STC at NOAA/JPSS (rebekah.esmaili@noaa.gov) and Dr. Amy Huff, IMSG at NOAA/NESDIS/STAR (amy.huff@noaa.gov). It demonstrates how to work with an ABI Level 2 netCDF4 file, including how to work with the ABI fixed grid/GOES Imager projection and what aspects to consider when making beautiful composite color (RGB) imagery.

The main steps are:
- Open the file
- Read the global file metadata & the metadata for variables in the file
    - Recognize the GOES Imager fixed grid projection variables
- Combine ABI band data variables to make different types of composite RGB imagery
- Plot composite RGB imagery using the native geostationary map projection
- Save image file

## Topic 1: Getting Started with Jupyter Notebook

### Step 1.1: Import Python packages

We will use four Python packages (libraries) and two Python modules in this tutorial:
- The **xarray** library is used to work with labelled multi-dimensional arrays
- The **NumPy** library is used to perform array operations
- The **Matplotlib** library is used to make plots
- The **Cartopy** library is used to create maps
- The **datetime** module is used to manipulate dates and times
- The **pathlib** module is used to set filesystem paths for the user's operating system

In [ ]:
import xarray as xr

import numpy as np

from matplotlib import pyplot as plt

from cartopy import crs as ccrs
import cartopy.feature as cfeature

import datetime

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

### Step 1.2: Set directory path for satellite data file

We set the directory path for the satellite data files using the [pathlib module](https://docs.python.org/3/library/pathlib.html#module-pathlib), which automatically uses the correct format for the user's operating system. This helps avoid errors in situations when more than one person is using the same code file, because Windows uses back slashes in directory paths, while MacOS and Linux use forward slashes. 

To keep things simple for this training, we put the satellite data files we downloaded in the current working directory (```Path.cwd()```), i.e., the same Jupyter Notebook folder where this code file is located.

In [ ]:
directory_path = Path.cwd()

## Topic 3: Understanding the Structure & Contents of netCDF Data Files

### Step 3.1: Open an ABI MCMIP satellite data file using xarray & read metadata

Let's use **xarray** to open one of the VIIRS fires data files we downloaded (```file_name```). We set the full path for the data file (```file_id```) using **pathlib** syntax.

We open the satellite data file using ```xr.open_dataset``` and then print the file metadata. The contents of a satellite data file are called a "Dataset" in **xarray**, conventionally abbreviated as ```ds```. 

The global file metadata are listed under ```Attributes```.

For any of the ```Data variables``` or ```Coordinates```, click on the attributes icon to see array metadata and the data repository icon to see an array summary.

The satellite data in the file are displayed under "Data variables".  A data variable is called a "DataArray" in **xarray**, conventionally abbreviated as ```da```. We are going to focus on the Cloud and Moisture Imagery (CMI) for the 16 ABI bands, also called channels (C); for example, ```CMI_C01``` is the CMI reflectance value for band (channel) 1.  We will also need the information in the ```goes_imager_projection``` data variable, which are constants that describe the fixed ABI grid for the GOES-16 satellite geostationary projection.

Notice that there are no arrays for latitude or longitude in the ```Coordinates```.  To save file space, ABI L1b and L2 data files do not contain latitude and longitude. 

In [ ]:
file_name = 'OR_ABI-L2-MCMIPM2-M6_G16_s20223362030552_e20223362031017_c20223362031113.nc'
file_id = directory_path / file_name

ds = xr.open_dataset(file_id, engine='netcdf4')
ds

### Step 3.2: Print ABI channel 1 metadata

The GOES-16 ABI instrument has [16 bands or channels](https://www.goes-r.gov/mission/ABI-bands-quick-info.html). Each channel has a central wavelength that were selected to target certain atmospheric phenomena. For example, Channel 1 has a central wavelength of 0.47 µm. In the file, the channels are abbreviated as ```CMI_C01```, ```CMI_C02```, ..., ```CMI_C16```. 

Let's look at the attributes of band 1 to learn more about the variable. Note, units set to '1' means the variable is a unitless quantity. 

In [ ]:
ds.CMI_C01.attrs

### Step 3.3: Print ABI channel 13 metadata

From the ```long_name```, we learned that the data are reflectances for band 1. Reflectance is the amount quantity of incoming radiation at that particular wavelength that is reflected back up and ranges from 0.0 (absorbing) to 1.0 (reflective). Reflectance is a common variable for visible and near-IR bands (ABI channels 1-6) whereas IR bands (ABI channels 7-16) are reported in brightness temperature, which is expressed in Kelvin.

In [ ]:
ds.CMI_C13.attrs

## Topic 4: Handling Data Arrays

### Overview: How to create an RGB plot

To make an red-green-blue (RGB) composite plot, we will need to define three arrays, one for each color. We can use the ```imshow``` function within **Matplotlib** to display an image of 3 two-dimensional arrays, each representing red, green, and blue.

Each of these arrays must:
* Have data values that span 0.0-1.0
* Be two-dimensional and the same size
* Be stacked "horizontally" or on the third dimension. The shape of the final array will be (X,Y,3)

### Step 4.1: Explore a simple example of an RGB plot

Let's explore a simple example of how to create an RGB plot. We'll populate 2x2 arrays with fake values.

In [ ]:
# top left | top right | bottom left | bottom right
red   = np.array( [ [ 1.0, 0.0 ], [ 0.0, 1.0] ] )
green = np.array( [ [ 0.0, 1.0 ], [ 0.0, 0.0] ] )
blue  = np.array( [ [ 0.0, 0.0 ], [ 1.0, 1.0] ] )

rgb = np.dstack([red, green, blue])

print(rgb.shape)

#### Step 4.1.1: Plot example data

```np.dstack()``` concatenates the given arrays horizontally. This function is also equivalent to ```np.stack([red, green,blue], axis=2)```. A quick check of the shape above shows it meets the shape requirements (X,Y,3) and the data we entered into the arrays is between 0 and 1. If the data are outside of that range, they are automatically "clipped" 1.0. It's best to manually scale the data though to make sure you're getting the results that you expect.

In [ ]:
plt.figure()
plt.imshow(rgb)
plt.show()

### Step 4.2: Check minimum and maximum values in the ABI channel 1 data array

Let's check the values in one of the channels in our ABI satellite data array. Do the min and max lie within 0 and 1?

In [ ]:
ds['CMI_C01'].values.min(), ds['CMI_C01'].values.max()

### Step 4.3: Check dimensions of data arrays in ABI MCMIP data file

Next, let's check the dimensions of the individual data arrays in the ABI MCIMIP file. In this file, all the data arrays are the same size (this is not true for all files!). Let's compare channels 1, 2, and 3.

We'll stack the three channels and check the shape of the stacked array in the next section!

In [ ]:
ds['CMI_C01'].shape, ds['CMI_C02'].shape, ds['CMI_C03'].shape

#### Step 4.4.1: How to change the size of data arrays

In this file, the data arrays all have the same dimensions (500, 500), so we can continue. If the arrays were not the same size, then some or all of the data would need to be re-gridded. For illustrative purposes, let's say we want to coarsen channel 1 by a factor of 2. We could change the size using **numpy** indexing:

In [ ]:
ch1 = ds['CMI_C01']
ch1_resize = ch1[::2,::2]
ch1.shape, ch1_resize.shape

A similar method would be to use **xarray's** ```coarsen``` function. The code below reduces the resolution by a factor of 2 and takes the mean of the original values.

In [ ]:
ch1_coarsen = ch1.coarsen(x=2, y=2).mean()
ch1_coarsen.shape

## Topic 5: Making Composite (RGB) Images

### True Color RGB Overview

True color imagery has delighted both scientists and the public since the beginning of the satellite age. These images are constructed from three layers, which respectively are red, green, and blue. To learn more, check out the [True Color RGB quick guide](https://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ABIQuickGuide_CIMSSRGB_v2.pdf) from Colorado State University's Cooperative Institute for Research in the Atmosphere.

For the GOES-16 ABI, we'll need the following "recipe" to make a True Color image:

|Color|central wavelength (µm)|channel | min - max (reflectance) | gamma |
|----|---|---|---|---|
|Red |0.64|2|0-1|1|
|Green|0.45\*Red + 0.1\*Veggie + 0.45\*Blue|2,3,1|0-1|1|
|Blue|0.47|1|0-1|1|

### Step 5.1: Extract ABI channel 1, 2, and 3 data arrays

The GOES-R series ABI sensor does not have a channel that detects the green wavelengths. Instead, it can be derived from a linear relationship between the red, blue, and another nicknamed the veggie channel. Let's extract these data arrays from our dataset.

In [ ]:
ch1 = ds.CMI_C01
ch2 = ds.CMI_C02
ch3 = ds.CMI_C03

### Step 5.2: Linearly combine the 3 arrays
As we discussed in Step 3.3, since these are in the visible/Near IR bands, the values are reflectances and thus unitless; they already span 0-1. We also already checked the shape and size in Step 4.3. If the arrays were not the same size, the next step would return an error. 

Use the formula in the table above to linearly combine the three arrays:

In [ ]:
green = 0.45*ch2 + 0.1*ch3 + 0.45*ch1

### Step 5.3: Check the range of green channel values

Is this new green channel still in range? Let's check:

In [ ]:
green.values.min(), green.values.max()

### Step 5.4: Combine (stack) the three data arrays

We can use ```np.dstack``` to combine the three arrays into a single 3D array. We must stack in order of red, green, blue layers:

In [ ]:
tc_RGB = np.dstack([ch2, green, ch1])

### Step 5.5: Make a quick plot using Matplotlib

Let's make a quick plot using ```plt.imshow```. Each of the three arrays will be respectively converted into red, green, and blue to create a color image.

In [ ]:
plt.figure()
plt.imshow(tc_RGB)
plt.show()

### Step 5.6: Adjust image luminance using gamma correction

We can see some features in the scene such as clouds and the underlying surface. However, the above image is a quite dark. This can be adjusted using an image gamma correction. Gamma adjusts the luminance in images. It can be calculated from the formula:

$$ layer_{adj} = { layer^{1 \over \gamma} } $$

Where gamma is expressed as ($\gamma$). Higher gamma values will brighten that color array while lower ones will darken. Let's arbitrarily pick a gamma of 2 to increase the brightness of the image:

In [ ]:
gamma = 2
tc_RGB_gamma = np.power(tc_RGB, 1/gamma)

plt.figure()
plt.imshow(tc_RGB_gamma)
plt.show()

### Dust RGB Overview

Just because we're working in RGB space doesn't limit us to only using the red, green, blue channels. We can combine any of the ABI channels to produce imagery. Since some channels are more sensitive to atmospheric constituents than others, RGB images can help distinguish natural features. We can also employ channel differencing to highlight specific atmospheric features.

Let's create a dust RGB image of the same scene next. The recipe below comes from CIRA's [Dust RGB quick guide](https://rammb.cira.colostate.edu/training/visit/quick_guides/Dust_RGB_Quick_Guide.pdf).

|color layer|central wavelength (µm)|channel | min - max (K) | gamma |
|----|---|---|---|---|
|Red |12.3 - 10.3|15 - 13| -6.7 to 2.6 |1|
|Green|11.2 - 8.2|14 - 11| -0.5 to 20.0 |2.5|
|Blue|10.3 |13 | 261.2 to 288.7 |1|

### Step 5.7: Extract ABI channel 11, 13, 14, and 15 data arrays

Just like before, let's extract the channels we need:

In [ ]:
ch11 = ds.CMI_C11
ch13 = ds.CMI_C13
ch14 = ds.CMI_C14
ch15 = ds.CMI_C15

### Step 5.8: Find ABI channel 15-13 difference for red layer

From the row describing the red layer in the table above, we will need to take the difference of channels 15 and 13:

In [ ]:
img = ch15-ch13

### Step 5.9: Normalize Red layer values

Because we are using ABI IR bands, the data are in brightness temperature. Thus, we will need to normalize the data [using the given min max ranges](https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)) so that it's on a 0 to 1 scale. This will look like:

$$ X_{norm} = {X - X_{min} \over X_{max} - X_{min} } $$

In [ ]:
lower_val = -6.7
upper_val = 2.6

# Sets all data above 2.6 to 1.0 and all data below -6.7 to 0:
img_clip = np.clip(img, lower_val, upper_val)
# Normalizes the clipped data
normalized_red = (img_clip-lower_val)/(upper_val-lower_val)

### Step 5.10: Check range of normalized Red layer values

Let's check if these values are in range:

In [ ]:
normalized_red.values.min(), normalized_red.values.max()

### Step 5.11: Define Green layer values and normalize

According to our dust RGB recipe, the gamma for the red layer is 1 and thus does not need to be changed. So, our red layer is now complete. 

Let's move on to the green layer:

In [ ]:
img = ch14-ch11

lower_val = -0.5
upper_val = 20.0

img_clip = np.clip(img, lower_val, upper_val)
normalized_green = (img_clip-lower_val)/(upper_val-lower_val)

### Step 5.12: Apply gamma correction to Green layer

The green layer requires requires a gamma correction:

In [ ]:
gamma = 2.5
normalized_green_gamma = np.power(normalized_green, 1/gamma)

### Step 5.13: Define Blue layer and normalize

This last layer uses a single channel with adjusted ranges. According to the third row of our recipe, no gamma correction is needed.

In [ ]:
img = ch13

lower_val = 261.2
upper_val = 288.7

img_clip = np.clip(img, lower_val, upper_val)
normalized_blue = (img_clip-lower_val)/(upper_val-lower_val)

### Step 5.14: Stack the Red, Green, and Blue layers

Just like in Step 5.4, We use ```np.dstack``` to combine the three arrays into a single 3D array. We stack in order of red, green, blue.

In [ ]:
dust_RGB = np.dstack([normalized_red, normalized_green_gamma, normalized_blue])

### Step 5.15: Plot dust RGB composite using Matplotlib

Below, we create a simple dust RGB plot:

In [ ]:
plt.figure()
plt.imshow(dust_RGB)
plt.show()

### Exercise RGB-1: Create an Airmass RGB composite image

[Airmass RGB](https://rammb.cira.colostate.edu/training/visit/quick_guides/QuickGuide_GOESR_AirMassRGB_final.pdf) composite images are used by meteorologists to easily display regions of polar and tropical air masses. Below is the recipe:

|Color|central wavelength (µm)|channel | min - max (K) | gamma |
|----|---|---|---|---|
|Red |6.2 – 7.3|8, 10 |-26.2 to 0.6 |1|
|Green|9.6 – 10.3|12, 13| -43.2 to 6.7 |1|
|Blue|6.2 (inverted)|8|244.0, 208.5|1|

Note: By inverted, we mean once the data are scaled to 0-1, you subtract 1-blue. So, low values (0) become high values (1), and vice versa.

Following the same steps we used for the RGB and dust RGB composites, construct an airmass RGB composite image below; some of the code is provided.

In [ ]:
# Define channels
ch8 = 
ch10 = 
ch12 = 
ch13 = 

In [ ]:
# Red
img = 

lower_val = 
upper_val = 

img_clip = np.clip(img, lower_val, upper_val)
normalized_red = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Green
img = 

lower_val = 
upper_val = 

img_clip = np.clip(img, lower_val, upper_val)
normalized_green = (img_clip-lower_val)/(upper_val-lower_val)

In [ ]:
# Blue
img = 

lower_val = 
upper_val = 

img_clip = np.clip(img, lower_val, upper_val)
normalized_blue = (img_clip-lower_val)/(upper_val-lower_val)
normalized_blue_inverted = 1-normalized_blue

In [ ]:
airmass_RGB = np.dstack([normalized_red, normalized_green, normalized_blue_inverted])

In [ ]:
plt.figure()
plt.imshow(airmass_RGB)
plt.show()

## Topic 6: Working with Map Projections

### Step 6.1: ABI fixed grid geostationary map projection constants

Now we can proceed to plotting our RGB composite images on a map. We will set the map projection for our plot using **Cartopy**.

**Cartopy** has many different [map projection options](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html), each with its own strengths and limitations. The VIIRS AOD notebook (viirs_aerosol.ipynb) walks through several examples of different map projections in greater detail.

We noticed in Topic 3 that this file does not have latitude and longitude as data variables; many of the GOES ABI products are defined on a fixed geostationary grid. We can extract information about the GOES satellite orbit directly from the netcdf file, which in turn can be used by **Cartopy** to project the data onto a geostationary grid.

In [ ]:
proj_var = ds.goes_imager_projection

sat_height = proj_var.perspective_point_height
semi_major = proj_var.semi_major_axis
semi_minor = proj_var.semi_minor_axis

globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)

### Step 6.2: Define the native geostationary map projection

The plotting function argument ```transform=ccrs.Geostationary()``` tells **Cartopy** that the RGB data are in geostationary coordinates. This argument **must** be included when plotting satellite data that are in geostationary coordinates, or the data will not plot correctly on the map projection.

Recall that geostationary satellites are always looking at a fixed point on the Earth's surface. The fixed point and satellite height are different on GOES-16/-17/-18 and on international partner satellites, like Meteosat and Himawari. The central latitude is included in the ABI data file, but we didn't include it below. This is because all current GOES satellites are centered on the equator, and the default value for ```central_latitude``` in the function is 0.

In [ ]:
central_lon = proj_var.longitude_of_projection_origin
crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)

### Step 6.3: Define extent of the image

It's also important to define the extent of the image. This tells **Cartopy** the boundaries of the image (versus that of the map). We do this by multiplying the x and y values (which is essentially the viewing angle of the satellite) by the height of the satellite. The smallest and largest values of each show the extent.

In [ ]:
X = ds['x']*sat_height
Y = ds['y']*sat_height
imgExtent = (X.min(), X.max(), Y.min(), Y.max())

### Step 6.4: Plot dust RGB composite on a geostationary map projection using Matplotlib & Cartopy

Finally, let's make a plot! You can switch the projection to Plate Carree by commenting out the second line. 

In [ ]:
proj_to = crs
# proj_to = ccrs.PlateCarree()

fig = plt.figure()
ax = plt.subplot(projection=proj_to)

ax.coastlines('10m', linewidth=2)
ax.imshow(dust_RGB, origin='upper', extent=imgExtent, transform=crs)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)

plt.show()

## Step 7: Adding Professional Touches to Images

### Step 7.1: Use information in the satellite data file metadata to create plot title

We can use the information in the global metadata for the satellite data file to create a plot title automatically. Let's print the global attributes to see what information we want to include in the plot title.

In [ ]:
ds.attrs

### Step 7.2: Extract information from global metadata and format plot title

It's good practice to include in the plot title the "formula" of the channels used to make the RGB. Recipes can vary slightly from satellite-to-satellite because:
* Not all sensors have the same central bands/channel numbering system
* Conventions can vary slightly between agencies, universities, and regions.

Mathematical symbols and formulas in strings can be written using [LaTeX-like syntax](https://en.wikibooks.org/wiki/LaTeX/Mathematics).

In [ ]:
platform = ds.platform_ID + ' ' + ds.title[0:3]

dtinfo_s = ds.time_coverage_start[0:16].replace('T',' ')
dtinfo_e = ds.time_coverage_end[0:16].replace('T',' ')

dt_scan = datetime.datetime.strptime(dtinfo_s, '%Y-%m-%d %H:%M')
date_s = dt_scan.strftime('%d %b %Y')
time_s = dt_scan.strftime('%H:%M')

composite = 'Dust RGB Composite'
formula = r'$Red = BT_{12.3\mu m}-BT_{10.3\mu m} \ \ \ Green = BT_{11.2\mu m}-BT_{8.4\mu m} \ \ \ Blue = BT_{10.3\mu m}$'

plot_title = platform + ' ' + composite + ' ' + time_s + '\n' + formula

### Step 7.3: Print plot title to check formatting

It's a good idea to do a quick check of the formula before adding it to the plot:

In [ ]:
plot_title

### Step 7.4: Add automatically generated plot title to map of dust RGB

In [ ]:
proj_to = crs
# proj_to = ccrs.PlateCarree()

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(projection=proj_to)

ax.coastlines('10m', linewidth=2)
ax.imshow(dust_RGB, origin='upper', extent=imgExtent, transform=crs)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)

plt.title(plot_title)

plt.show()